In [1]:
'''
UHI Index: (Temperature at a given location) / (Mean temperature for all locations) (Symmetrical Distribution)
MAX: 1.046; MIN: 0.956

predict heat island hotspots in an urban location by using satellite data; regression model

use building footprints to perhaps identify relationship between building congestion and heat
use optical satellite data to identify relationship between water bodies and greeneries, relative to heat

green space; sentinel 2 data, satellite data, and common vegetation indices 
can't use lat and long as features directly. possible to generate attributes to the data, ie. distance from buildings, proximity to green space

y: UHI index
x: multispectral bands, indices like NDVI from sentinel-2(from bands B04 and B08)

preprocessing: uncover patterns driving UHI, explore algorithms like random forest

microsoft planetary computer portal; explore satellite datasets, including sentinel-2, landsat and more

can create own geoTIFF files to extract band values from satellite data
or use API calling methods

possible areas to improve model:
1. using different combination of bands from Sentinel-2 and other datasets; apply mathematical operations on the bands to generate various indices
2. considering using a larger time period rather than focusing on bands from a single day; explore a larger time period to generate time series statistics for bands and indices
3. create a buffer around locations to calculate average band values, reducing errors from spatial autocorrelation.(eg, 50m, 100m, 150m)
4. Explore different data preprocessing methods and machine learning algorithms to enhance model performance
'''

'\nUHI Index: (Temperature at a given location) / (Mean temperature for all locations) (Symmetrical Distribution)\nMAX: 1.046; MIN: 0.956\n'

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

# Multi-dimensional arrays and datasets
import xarray as xr

# Geospatial raster data handling
import rioxarray as rxr

# Geospatial data analysis
import geopandas as gpd

# Geospatial operations
import rasterio
from rasterio import windows  
from rasterio import features  
from rasterio import warp
from rasterio.warp import transform_bounds 
from rasterio.windows import from_bounds 

# Image Processing
from PIL import Image

# Coordinate transformations
from pyproj import Proj, Transformer, CRS

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Planetary Computer Tools
import pystac_client
import planetary_computer as pc
from pystac.extensions.eo import EOExtension as eo

# Others
import os
from tqdm import tqdm

In [3]:
df = pd.read_csv("/Users/kanghong/Documents/kang's codespace/EY Challenge 2025/Training_data_uhi_index_2025-02-18.csv")
df.head()

,Longitude,Latitude,datetime,UHI Index
0,-73.909167,40.813107,24-07-2021 15:53,1.030289
1,-73.909187,40.813045,24-07-2021 15:53,1.030289
2,-73.909215,40.812978,24-07-2021 15:53,1.023798
3,-73.909242,40.812908,24-07-2021 15:53,1.023798
4,-73.909257,40.812845,24-07-2021 15:53,1.021634


In [7]:
df.describe()

,Longitude,Latitude,UHI Index
count,11229.000000,11229.000000,11229.000000
mean,-73.933927,40.808800,1.000001
std,0.028253,0.023171,0.016238
min,-73.994457,40.758792,0.956122
25%,-73.955703,40.790905,0.988577
50%,-73.932968,40.810688,1.000237
75%,-73.909647,40.824515,1.011176
max,-73.879458,40.859497,1.046036
